call functions:
1. '\show': Shows events within the month (please visit Teampup link for all activities planned)

auto trigger functions
1. at the start of every month shows events within the month
2. 1 week before & 1 day before the start of the event, message the event

notes:
bot will auto fetch the data every day. this means that any changes within the day will only be reflected the next day.

In [1]:
# https://github.com/eternnoir/pyTelegramBotAPI/
# pip install pyTelegramBotAPI
import telebot
import pandas as pd
from datetime import datetime,timedelta
import requests
import json

In [2]:
def msg(Calendar_Name, Subject, Start_Date, End_Date):
    # converting to more readable format
    if Start_Date.date() == End_Date.date():
        Date = Start_Date.date().strftime('%d %b %Y')
    else:
        Date = f"{Start_Date.date().strftime('%d %b %Y')} - {End_Date.date().strftime('%d %b %Y')}"
    # Sample message to be displayed
    return (f"""Section: {Calendar_Name}
Event: {Subject.title()}
Date: {Date}""")

In [3]:
headers = ({"Teamup-Token": "f82c44f68f2823ceec7b948e35dc7d9e13c83e24a84fce29d2f2c9d2fdd712a8"})
start_date = "2020-07-01"
end_date = "2021-08-01"

In [4]:
# To get the sub calendar id
# Commented out so as to not request from the TeamUp everytime AWS restarts

# sub_cal_json = requests.get("https://api.teamup.com/ksbrc8q6drw9ns4pgd/subcalendars", headers=headers).json()
# sub_calendar_id = {}
# for sub_calendar in sub_cal_json["subcalendars"]:
#     sub_calendar_id[sub_calendar["id"]] = sub_calendar["name"]

sub_calendar_id = {8247175: 'Alumni Relations',
                   8235037: 'Events',
                   8235039: 'External Relations',
                   8224178: 'Meeting/School Start/Recess Week',
                   8235109: 'NBS BAC',
                   8235041: 'Press & Publications',
                   8234993: 'Public Holidays/Revision & Examination',
                   8235045: 'Service Learning'}

In [5]:
events = requests.get(f"https://api.teamup.com/ks************/events?startDate={start_date}&endDate={end_date}", headers=headers).json()
# URL of TeamUp calendar. Removed for security reasons.

subject = []
s_date = []
e_date = []
calendar_name = []

for event in events["events"]:
    subject.append(event["title"])
    st_date = event["start_dt"]
    s_date.append(f"{st_date[8:10]}/{st_date[5:7]}/{st_date[0:4]}")
    en_date = event["end_dt"]
    e_date.append(f"{en_date[8:10]}/{en_date[5:7]}/{en_date[0:4]}")
    sub_cal_id = event["subcalendar_id"]
    calendar_name.append(sub_calendar_id[sub_cal_id])

df = pd.DataFrame({"Subject" : subject,
                  "Start Date" : s_date,
                  "End Date" : e_date,
                  "Calendar Name" : calendar_name})

In [6]:
# Reading of the file if export TeamUp calendar by .csv
# fields = ["Subject","Start Date","End Date", "Calendar Name"]
# only reads in the useful cols
# df = pd.read_csv("lol.csv", usecols = fields)

In [7]:
# Event types to be ignored
ignore = ["Public Holidays/Revision & Examination"]
df = df[~df["Calendar Name"].isin(ignore)]

# drop "Meeting/School Start/Recess Week" that isnt meeting
condition = (df["Calendar Name"] == "Meeting/School Start/Recess Week") & (df["Subject"] != "BAC Meeting")
df = df.loc[-condition,:]

# Converting the dates string into datetime format
df["Start Date"] = pd.to_datetime(df["Start Date"],format="%d/%m/%Y")
df["End Date"] = pd.to_datetime(df["End Date"],format="%d/%m/%Y")

# only keep if the date is later
current_date = datetime.now()
df = df[df["Start Date"]>current_date]
df = df.reset_index(drop=True)

Monthly Message

In [8]:
m_df = df.loc[df['Start Date'].dt.month==current_date.month,:].sort_values(["Start Date", "Calendar Name", "Subject"]).reset_index(drop=True)

if len(m_df) == 0:
    m_text = "There are no scheduled event for the month"
else:
    m_text = "Please be reminded that the following event is scheduled: \n"
    for r in range(len(m_df)):
        m_text += msg(m_df.loc[r, "Calendar Name"], m_df.loc[r, "Subject"], m_df.loc[r, "Start Date"], m_df.loc[r, "End Date"])
        m_text += "\n\n"

Weekly/Daily Message

In [9]:
index = [] # index of events which need to send reminders
for i in range(len(df["Start Date"])):
    t = df.loc[i,"Start Date"].date() - current_date.date()
    if t == timedelta(days=7) or t == timedelta(days=1): # No. of days given in advance
        index.append(i)
    
wd_df = df.loc[index, :].sort_values(["Start Date", "Calendar Name", "Subject"]).reset_index(drop=True)
wk_text = "Please be reminded that the following event is scheduled: \n"
for r in range(len(wd_df)):
    wk_text += msg(wd_df.loc[r, "Calendar Name"], wd_df.loc[r, "Subject"], wd_df.loc[r, "Start Date"], wd_df.loc[r, "End Date"])
    wk_text += "\n\n"

In [10]:
TOKEN = "" # telegram Bot Token. Removed for security reasons
CHAT_ID = "-413526424"# Chat room ID
bot = telebot.TeleBot(TOKEN)

In [11]:
# prints m_text when "/show @Reminder"
@bot.message_handler(commands=['show'])
def show(message):
    bot.reply_to(message, m_text)
    
# send monthly message
if current_date.day == 1:
    bot.send_message(chat_id = CHAT_ID, text = m_text)

# send weekly/daily message
if wk_text != "Please be reminded that the following event is scheduled: \n":
    bot.send_message(chat_id = CHAT_ID, text = wk_text)

bot.polling(none_stop=False, interval=0, timeout=20)

In [12]:
# print(wk_text)

In [13]:
# print(m_text)

In [14]:
# To host on AWS EC2: https://www.youtube.com/watch?v=WE303yFWfV4&list=LLXY2pMaIWQQfnyHO5RsO3wQ&index=31
#            or this: https://www.youtube.com/watch?v=BYvKv3kM9pk&list=LLXY2pMaIWQQfnyHO5RsO3wQ&index=31&t=0s
# sceduling of task: 2nd link, timestamp: 14:20

# commands:
# after starting session in Putty: type "screen -r"
# close "screen" but server still running: press "ctrl + A + D"
# stop server (don't do this): press "ctrl + c"